<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/%20https:/cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/images/IDSNlogo.png" width="300" alt="cognitiveclass.ai logo">
</center>


# **Launch Sites Locations Analysis with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [1]:
!pip install folium==0.7.0
!pip3 install wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: folium
    Found existing installation: folium 0.5.0
    Uninstalling folium-0.5.0:
      Successfully uninstalled folium-0.5.0


In [2]:
import folium
import wget
import pandas as pd

In [3]:
from folium import plugins

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module\_3/DV0101EN-3-5-1-Generating-Maps-in-Python-py-v2.0.ipynb)


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('Nasa JSC'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'Nasa JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [9]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=4)
launches = folium.map.FeatureGroup()

latitudes = list(launch_sites_df.Lat)
longitudes = list(launch_sites_df.Long)
labels = list (launch_sites_df['Launch Site'])

for lat, lng, label in zip(latitudes, longitudes, labels):
    circle = folium.Circle([lat,lng], radius=500, color='#d35400', fill=True).add_child(folium.Popup(label))
    marker = folium.map.Marker([lat,lng],icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0),html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % label))
    #site_map.add_child(folium.Popup(site_name))
    site_map.add_child(circle)
    site_map.add_child(marker)
site_map.add_child(launches)

The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png" />
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


# Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [10]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [11]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Function to assign color to launch outcome
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
spacex_df.tail(10)

,Launch Site,Lat,Long,class,marker_color
46,KSC LC-39A,28.573255,-80.646895,1,green
47,KSC LC-39A,28.573255,-80.646895,1,green
48,KSC LC-39A,28.573255,-80.646895,1,green
49,CCAFS SLC-40,28.563197,-80.576820,1,green
50,CCAFS SLC-40,28.563197,-80.576820,1,green
51,CCAFS SLC-40,28.563197,-80.576820,0,red
52,CCAFS SLC-40,28.563197,-80.576820,0,red
53,CCAFS SLC-40,28.563197,-80.576820,0,red
54,CCAFS SLC-40,28.563197,-80.576820,1,green
55,CCAFS SLC-40,28.563197,-80.576820,0,red


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [12]:
launches_cluster = plugins.MarkerCluster().add_to(site_map)

for lat, lng, label, col in zip(spacex_df.Lat,spacex_df.Long, spacex_df['Launch Site'],spacex_df['marker_color']):
    folium.Marker(location=[lat, lng], icon = folium.Icon(color=col), popup=label).add_to(launches_cluster)
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png" />
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png" />
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [13]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


You can calculate the distance between two points on the map based on their `Lat` and `Long` values using the following method:


In [14]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [15]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

launch_site_coordinates = (28.56316, -80.57681)
coastline_coordinates = (28.56318, -80.56792)
distance_coastline = calculate_distance(launch_site_coordinates[0], launch_site_coordinates[1], coastline_coordinates[0], coastline_coordinates[1])
distance_coastline

0.8684856692847999

*TODO:* After obtained its coordinate, create a `folium.Marker` to show the distance


In [16]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
# for example
distance_marker = folium.Marker(
    location=coastline_coordinates,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
        )
    ).add_to(site_map)
site_map

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [17]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
points = [coastline_coordinates , launch_site_coordinates]
lines=folium.PolyLine(points, color = '#d35400', weight=2).add_to(site_map)
site_map.add_child(lines)
site_map

Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png" />
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png" />
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png" />
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png" />
</center>


# KSC LC-39A

In [18]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_coordinates_1 = (28.5733, -80.64685)
coastline_coordinates_1 = (28.59938, -80.58616)
railway_coordinates_1 = (28.57141, -80.65387)
highway_coordinates_1 = (28.57324, -80.65549)
city_coordinates_1 = (28.61244, -80.80745)
distance_coastline_1 = calculate_distance(launch_site_coordinates_1[0], launch_site_coordinates_1[1], coastline_coordinates_1[0], coastline_coordinates_1[1])
distance_railway_1 = calculate_distance(launch_site_coordinates_1[0], launch_site_coordinates_1[1], railway_coordinates_1[0], railway_coordinates_1[1])
distance_highway_1 = calculate_distance(launch_site_coordinates_1[0], launch_site_coordinates_1[1], highway_coordinates_1[0], highway_coordinates_1[1])
distance_city_1 = calculate_distance(launch_site_coordinates_1[0], launch_site_coordinates_1[1], city_coordinates_1[0], city_coordinates_1[1])

In [19]:
distance_marker_1 = folium.Marker(
    location=coastline_coordinates_1,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_1),
        )
    ).add_to(site_map)

distance_marker_2 = folium.Marker(
    location=railway_coordinates_1,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_1),
        )
    ).add_to(site_map)

distance_marker_3 = folium.Marker(
    location=highway_coordinates_1,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_1),
        )
    ).add_to(site_map)

distance_marker_4 = folium.Marker(
    location=city_coordinates_1,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_1),
        )
    ).add_to(site_map)

points_1 = [coastline_coordinates_1 , launch_site_coordinates_1]
points_2 = [railway_coordinates_1 , launch_site_coordinates_1]
points_3 = [highway_coordinates_1 , launch_site_coordinates_1]
points_4 = [city_coordinates_1 , launch_site_coordinates_1]

lines_1=folium.PolyLine(points_1, color = '#d35400', weight=2).add_to(site_map)
lines_2=folium.PolyLine(points_2, color = '#d35400', weight=2).add_to(site_map)
lines_3=folium.PolyLine(points_3, color = '#d35400', weight=2).add_to(site_map)
lines_4=folium.PolyLine(points_4, color = '#d35400', weight=2).add_to(site_map)

site_map.add_child(lines_1)
site_map.add_child(lines_2)
site_map.add_child(lines_3)
site_map.add_child(lines_4)

site_map

# CCAFS SLC-40

In [20]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_coordinates_2 = (28.56316, -80.57681)
coastline_coordinates_2 = (28.56318, -80.56792)
railway_coordinates_2 = (28.57498, -80.58511)
highway_coordinates_2 = (28.56429, -80.57092)
city_coordinates_2 = (28.40311, -80.5976)
distance_coastline_2 = calculate_distance(launch_site_coordinates_2[0], launch_site_coordinates_2[1], coastline_coordinates_2[0], coastline_coordinates_2[1])
distance_railway_2 = calculate_distance(launch_site_coordinates_2[0], launch_site_coordinates_2[1], railway_coordinates_2[0], railway_coordinates_2[1])
distance_highway_2 = calculate_distance(launch_site_coordinates_2[0], launch_site_coordinates_2[1], highway_coordinates_2[0], highway_coordinates_2[1])
distance_city_2 = calculate_distance(launch_site_coordinates_2[0], launch_site_coordinates_2[1], city_coordinates_2[0], city_coordinates_2[1])

In [21]:
distance_marker_5 = folium.Marker(
    location=coastline_coordinates_2,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_2),
        )
    ).add_to(site_map)

distance_marker_6 = folium.Marker(
    location=railway_coordinates_2,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_2),
        )
    ).add_to(site_map)

distance_marker_7 = folium.Marker(
    location=highway_coordinates_2,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_2),
        )
    ).add_to(site_map)

distance_marker_8 = folium.Marker(
    location=city_coordinates_2,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_2),
        )
    ).add_to(site_map)

points_5 = [coastline_coordinates_2 , launch_site_coordinates_2]
points_6 = [railway_coordinates_2 , launch_site_coordinates_2]
points_7 = [highway_coordinates_2 , launch_site_coordinates_2]
points_8 = [city_coordinates_2 , launch_site_coordinates_2]

lines_5=folium.PolyLine(points_5, color = '#d35400', weight=2).add_to(site_map)
lines_6=folium.PolyLine(points_6, color = '#d35400', weight=2).add_to(site_map)
lines_7=folium.PolyLine(points_7, color = '#d35400', weight=2).add_to(site_map)
lines_8=folium.PolyLine(points_8, color = '#d35400', weight=2).add_to(site_map)

site_map.add_child(lines_5)
site_map.add_child(lines_6)
site_map.add_child(lines_7)
site_map.add_child(lines_8)

site_map

# CCAFS LC-40

In [22]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_coordinates_3 = (28.56231, -80.57737)
coastline_coordinates_3 = (28.55995, -80.56745)
railway_coordinates_3 = (28.5721, -80.58547)
highway_coordinates_3 = (28.56135, -80.57033)
city_coordinates_3 = (28.40489, -80.60429)
distance_coastline_3 = calculate_distance(launch_site_coordinates_3[0], launch_site_coordinates_3[1], coastline_coordinates_3[0], coastline_coordinates_3[1])
distance_railway_3 = calculate_distance(launch_site_coordinates_3[0], launch_site_coordinates_3[1], railway_coordinates_3[0], railway_coordinates_3[1])
distance_highway_3 = calculate_distance(launch_site_coordinates_3[0], launch_site_coordinates_3[1], highway_coordinates_3[0], highway_coordinates_3[1])
distance_city_3 = calculate_distance(launch_site_coordinates_3[0], launch_site_coordinates_3[1], city_coordinates_3[0], city_coordinates_3[1])

In [23]:
distance_marker_9 = folium.Marker(
    location=coastline_coordinates_3,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_3),
        )
    ).add_to(site_map)

distance_marker_10 = folium.Marker(
    location=railway_coordinates_3,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_3),
        )
    ).add_to(site_map)

distance_marker_11 = folium.Marker(
    location=highway_coordinates_3,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_highway_3),
        )
    ).add_to(site_map)

distance_marker_12 = folium.Marker(
    location=city_coordinates_3,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_3),
        )
    ).add_to(site_map)

points_9 = [coastline_coordinates_3 , launch_site_coordinates_3]
points_10 = [railway_coordinates_3 , launch_site_coordinates_3]
points_11 = [highway_coordinates_3 , launch_site_coordinates_3]
points_12 = [city_coordinates_3 , launch_site_coordinates_3]

lines_9=folium.PolyLine(points_9, color = '#d35400', weight=2).add_to(site_map)
lines_10=folium.PolyLine(points_10, color = '#d35400', weight=2).add_to(site_map)
lines_11=folium.PolyLine(points_11, color = '#d35400', weight=2).add_to(site_map)
lines_12=folium.PolyLine(points_12, color = '#d35400', weight=2).add_to(site_map)

site_map.add_child(lines_9)
site_map.add_child(lines_10)
site_map.add_child(lines_11)
site_map.add_child(lines_12)

site_map

# VAFB SLC-4E

In [24]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
launch_site_coordinates_4 = (34.63271, -120.6109)
coastline_coordinates_4 = (34.62924, -120.62874)
railway_coordinates_4 = (34.63468, -120.62428)
city_coordinates_4 = (34.63881, -120.45981)
distance_coastline_4 = calculate_distance(launch_site_coordinates_4[0], launch_site_coordinates_4[1], coastline_coordinates_4[0], coastline_coordinates_4[1])
distance_railway_4 = calculate_distance(launch_site_coordinates_4[0], launch_site_coordinates_4[1], railway_coordinates_4[0], railway_coordinates_4[1])
distance_city_4 = calculate_distance(launch_site_coordinates_4[0], launch_site_coordinates_4[1], city_coordinates_4[0], city_coordinates_4[1])

In [25]:
distance_marker_13 = folium.Marker(
    location=coastline_coordinates_4,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline_4),
        )
    ).add_to(site_map)

distance_marker_14 = folium.Marker(
    location=railway_coordinates_4,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_railway_4),
        )
    ).add_to(site_map)

distance_marker_15 = folium.Marker(
    location=city_coordinates_4,
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_city_4),
        )
    ).add_to(site_map)

points_13 = [coastline_coordinates_4 , launch_site_coordinates_4]
points_14 = [railway_coordinates_4 , launch_site_coordinates_4]
points_15 = [city_coordinates_4 , launch_site_coordinates_4]

lines_13=folium.PolyLine(points_13, color = '#d35400', weight=2).add_to(site_map)
lines_14=folium.PolyLine(points_14, color = '#d35400', weight=2).add_to(site_map)
lines_15=folium.PolyLine(points_15, color = '#d35400', weight=2).add_to(site_map)

site_map.add_child(lines_13)
site_map.add_child(lines_14)
site_map.add_child(lines_15)

site_map

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


### The launch sites CCAFS LC-40 CCAFS SLC-40 are in close proximity to railways, highways and coastline (less than 2 Km); The launc site KSC LC-39A is in close proximity to railways and highways (less than 1 Km) and has a distance fron coastline of about 7 Km; The launch sire VAFB SLC-4E is in close proximity to railways and coastline (less than 2 Km) and it isn't in close proximity to highways. All the sites keep certain distance away from cities (more than 13 Km)

# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Yan Luo](https://www.linkedin.com/in/yan-luo-96288783/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01)


### Other Contributors


Joseph Santarcangelo


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description          |
| ----------------- | ------- | ---------- | --------------------------- |
| 2021-05-26        | 1.0     | Yan        | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
